In [10]:
import sys 

sys.path.append(r'C:\Users\DerDo\Desktop\fantasy_basketball_project')
from scrape_functions import *
from database_conn import SQLiteClient

scraper = Scrape_Functions()
db_path = r"D:\nba.db"

In [38]:
all_team_data = scraper.all_team_historical_data()

new_cols = [
    col.lower().replace('/', '').replace('%', '') for col in all_team_data.columns
]
new_cols[2] = 'start'
new_cols[3] = 'end'

all_team_data.columns = new_cols

all_team_data = all_team_data.iloc[
    all_team_data['franchise'].drop_duplicates(keep='first').index
].reset_index(drop=True)

# # Create or use the DB
with SQLiteClient(db_path) as db:
    # 1) Create a table (only once)
    db.create_table(
        "historical_team_data",
        {
            "franchise": "TEXT PRIMARY KEY",
            "lg": "TEXT NULL",
            "start": "TEXT NULL",
            "end": "TEXT NULL",
            "yrs": "TEXT NULL",
            "g": "TEXT NULL",
            "w": "TEXT NULL",
            "l": "TEXT NULL",
            "wl": "TEXT NULL",
            'plyfs': "TEXT NULL",
            'div': 'TEXT NULL',
            'conf': 'TEXT NULL',
            'champ': 'TEXT NULL'
        }
    )

    db.upsert_dataframe(df=all_team_data, table='historical_team_data', conflict_cols='franchise')

In [45]:
with SQLiteClient(db_path) as db:
    data = db.query('SELECT * FROM historical_team_data')

import pandas as pd
pd.DataFrame(data)

,franchise,lg,start,end,yrs,g,w,l,wl,plyfs,div,conf,champ
0,Atlanta Hawks,NBA,1949-50,2025-26,77,6019,2967,3052,.493,49,12,0,1
1,St. Louis Hawks,NBA,1955-56,1967-68,13,1005,553,452,.550,12,6,0,1
2,Milwaukee Hawks,NBA,1951-52,1954-55,4,281,91,190,.324,0,0,0,0
3,Tri-Cities Blackhawks,NBA,1949-50,1950-51,2,132,54,78,.409,1,0,0,0
4,Boston Celtics,NBA/BAA,1946-47,2025-26,80,6196,3695,2501,.596,62,35,11,18
...,...,...,...,...,...,...,...,...,...,...,...,...,...
57,Washington Bullets,NBA,1974-75,1996-97,23,1886,887,999,.470,13,2,3,1
58,Capital Bullets,NBA,1973-74,1973-74,1,82,47,35,.573,1,1,0,0
59,Baltimore Bullets,NBA,1963-64,1972-73,10,813,401,412,.493,7,4,1,0
60,Chicago Zephyrs,NBA,1962-63,1962-63,1,80,25,55,.313,0,0,0,0
